In [1]:
import torch 
from torch import nn
import os
os.environ["CUDA_VISIBLE_DEVICES"]='1'
# from alphagen.config import *
# from alphagen.data.tokens import *
from alphagen.models.alpha_pool import AlphaPoolBase, AlphaPool
from alphagen.rl.env.core import AlphaEnvCore
import torch.nn.functional as F
from gan.dataset import Collector
from gan.network.generater import NetG_DCGAN
from gan.network.masker import NetM
from gan.network.predictor import NetP, train_regression_model,train_regression_model_with_weight
from alphagen.rl.env.wrapper import SIZE_ACTION,action2token

from alphagen_generic.features import open_
from gan.utils import Builders
from alphagen_generic.features import *
from alphagen.data.expression import *

from gan.utils.data import get_data_by_year


2025-08-22 11:56:38.920756: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-22 11:56:38.979308: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/DATA1/home/chenbq/AlphaStruct/.venv/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/DAT

In [2]:

instruments: str = "csi300"
freq = 'day'
save_name = 'test'
window = float('inf')

In [10]:
from alphagen.utils.correlation import batch_pearsonr,batch_spearmanr
import pandas as pd
device = 'cuda:0'
result = []
pred_dfs = {}
for n_factors in [10]:#[1,10,20,50,100]:
    for seed in range(1):
        cur_seed_ic = []
        cur_seed_ric = []
        all_pred_df_list = []
        for train_end in range(2020,2021):
            print(n_factors,seed,train_end)
            returned = get_data_by_year(
                train_start = 2010,train_end=train_end,valid_year=train_end+1,test_year =train_end+2,
                instruments=instruments, target=target,freq=freq,
            )
            data_all, data,data_valid,data_valid_withhead,data_test,data_test_withhead,name = returned
            

            path = f'out/{save_name}_{instruments}_{train_end}_{seed}/z_bld_zoo_final.pkl'
            tensor_save_path = f'out/{save_name}_{instruments}_{train_end}_{seed}/pred_{train_end}_{n_factors}_{window}_{seed}.pt'

            pred = torch.load(tensor_save_path).to(device)
            tgt = target.evaluate(data_all)
            
            
            ones = torch.ones_like(tgt)
            ones = ones * torch.nan
            print(data_test._start_time, data_test._end_time)
            ones[-data_test.n_days:] = pred
            cur_df = data_all.make_dataframe(ones)
            all_pred_df_list.append(cur_df.unstack().iloc[-data_test.n_days:].stack())
            
            tgt = tgt[-data_test.n_days:].to(device)
            
            
            ic_s = torch.nan_to_num(batch_pearsonr(pred,tgt),nan=0)
            rank_ic_s = torch.nan_to_num(batch_spearmanr(pred,tgt),nan=0)

            cur_seed_ic.append(ic_s)
            cur_seed_ric.append(rank_ic_s)
            
        pred_dfs[f"{n_factors}_{seed}"] = pd.concat(all_pred_df_list,axis=0)
        ic = torch.cat(cur_seed_ic)
        rank_ic = torch.cat(cur_seed_ric)

        ic_mean = ic.mean().item()
        rank_ic_mean = rank_ic.mean().item()
        ic_std = ic.std().item()
        rank_ic_std = rank_ic.std().item()
        tmp = dict(
            seed = seed,
            num = n_factors,
            ic = ic_mean,
            ric = rank_ic_mean,
            icir = ic_mean/ic_std,
            ricir = rank_ic_mean/rank_ic_std,
        )
        result.append(tmp)

10 0 2020


/DATA1/home/chenbq/AlphaStruct/.venv/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))
/DATA1

2022-01-01 2024-12-31


/tmp/ipykernel_3981469/2906412734.py:32: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  all_pred_df_list.append(cur_df.unstack().iloc[-data_test.n_days:].stack())


In [11]:
import pandas as pd
run_result = pd.DataFrame(result).groupby(['num','seed']).mean().groupby('num').agg(['mean','std'])
print(run_result)

           ic           ric          icir         ricir    
         mean std      mean std      mean std      mean std
num                                                        
10   0.046395 NaN  0.052695 NaN  0.316842 NaN  0.343857 NaN
